In [ ]:

# # Challenges:
# * Encode companies
# * Encode people
# * Encode cities and countries
# * use text data embedding (word/document) or word freq 
# * Imputation

# imports

In [ ]:


import pandas as pd
import numpy as np
import requests
import plotly.express as px
from sklearn import preprocessing


# Reviewing a sample row from each file

In [ ]:


acquired = pd.read_csv("Data/Acquired Tech Companies.csv")
acquired.iloc[0]

In [ ]:


acquiring = pd.read_csv("Data/Acquiring Tech Companies.csv")
acquiring.iloc[0]

In [ ]:


acquisitions = pd.read_csv("Data/Acquisitions.csv")
acquisitions.iloc[0]

In [ ]:


founders = pd.read_csv("Data/Founders and Board Members.csv")
founders.iloc[0]


# We will link between the files using these columns:
# * Acquisitions ID to link the acquisitions
# * 'Founders' and 'Name' to link the Founders

In [ ]:


np.intersect1d(acquired.columns, acquisitions.columns).tolist()

In [ ]:


np.intersect1d(acquiring.columns, acquisitions.columns).tolist()

In [ ]:


def ValidateLink(url, timeout=15):
    session = requests.Session()
    # fake headers to make it seem like a real request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "DNT": "1",
    }
    session.headers.update(headers)
    try:
        response = session.get(url, timeout=timeout, allow_redirects=True, stream=True)
        status_code = response.status_code
        response.close()
        if status_code < 400:
            return True
        else:
            return False
    except Exception as e:
        return False

In [ ]:


def ValidateLinks(urls):
    results = []
    for url in urls:
        results.append(ValidateLink(url))
        if results[-1]:
            return results
    return results

In [ ]:


def ValidateLinksDF(df):
    for col in df.columns:
        for val in df[col]:
            if type(val) == str and ("http" in val):
                print(col)
                results = ValidateLinks(df[col])
                if not pd.Series(results).any():
                    print(f'Column "{col}" had no valid links , or is using captcha.')
                    print("Try it yourself:")
                    print(df[col][0] + "\n")
                break


# ValidateLinksDF(acquired)

# * CrunchBase is using CAPTCHA , so we won't drop it now but we will process it later
# * Image links are all corrupt so we will drop the column 

In [ ]:


acquired = acquired.drop("Image", axis=1)
acquired = acquired.drop("Tagline", axis=1)


# ValidateLinksDF(acquiring)

# * drop Image also

In [ ]:


acquiring = acquiring.drop("Image", axis=1)


# ValidateLinksDF(acquisitions)

# * acquisitions profile is also a crunchbase link

In [ ]:


acquisitions = acquisitions.drop("Acquisition Profile", axis=1)


# ValidateLinksDF(founders)

# We don't need the exact address of the company, we already have the city , state and country

In [ ]:


acquired = acquired.drop("Address (HQ)", axis=1)
acquiring = acquiring.drop("Address (HQ)", axis=1)


# **Adding the target variable**

In [ ]:


acquisitions["Price"] = [
    int(price.removeprefix("$").replace(",", "")) for price in acquisitions["Price"]
]

In [ ]:


acquired["Price"] = None
acquired["Year of acquisition announcement"] = None

In [ ]:


for i, company in enumerate(acquisitions["Acquired Company"]):
    acquired.loc[acquired["Company"] == company, "Price"] = acquisitions.iloc[i][
        "Price"
    ]
    acquired.loc[acquired["Company"] == company, "Year of acquisition announcement"] = (
        acquisitions.iloc[i]["Year of acquisition announcement"]
    )

In [ ]:


fig = px.scatter(
    acquisitions,
    x="Year of acquisition announcement",
    y="Price",
    title="Acquisition Price by Year",
    width=600,
    height=400,
)
fig.show()


# There was a wrongly entered value, so I looked at the link and corrected it

In [ ]:


acquisitions.loc[
    acquisitions["Year of acquisition announcement"] == 2104,
    "Year of acquisition announcement",
] = 2014


# Plotting again without the error, now we can see that the overall trend of prices tends to go up, that's why we added the 'Year of acquisitions announcement' column

In [ ]:


fig = px.scatter(
    acquisitions,
    x="Year of acquisition announcement",
    y="Price",
    title="Acquisition Price by Year",
    width=700,
    height=400,
)
fig.show()


# update the datatypes automatically

In [ ]:


acquired = acquired.infer_objects()
acquisitions = acquisitions.infer_objects()

In [ ]:


fig = px.scatter(
    acquired,
    x="Year Founded",
    y="Price",
    title="Acquisition Price by Year",
    width=600,
    height=400,
)
fig.show()


# Another error found and corrected

In [ ]:


acquired.loc[acquired["Year Founded"] == 1840, "Year Founded"] = 2006
acquired.loc[acquired["Year Founded"] == 1933, "Year Founded"] = 1989

In [ ]:


fig = px.scatter(
    acquired,
    x="Year Founded",
    y="Price",
    title="Acquisition Price by Year",
    width=600,
    height=400,
)
fig.show()

In [ ]:


#acquired.iloc[12]["Tagline"]

In [ ]:


for l in acquired.iloc[12]["Description"].split("."):
    print(l + "\n")


# * 'Tagline' contains a brief and precise description of the company , while the 'Description' is very long and doesn't provide any more important details, 
# so we will drop the 'Description'

In [ ]:


acquiring = acquiring.drop("Description", axis=1)
acquired = acquired.drop("Description", axis=1)


# ### There isn't any new useful information that we can get out of those , so we will drop them

# * "CrunchBase Profile" and "API" columns are both on the crunchbase website , which uses captcha so we can't scrap it, and their API is paid , and the provided API key is invalid , so we can't use it
# 
# * "Homepage" column contains the link to the website of every company , and they aren't all the same so we can't apply a function or a program to extract certain information about them. To use the link , this would require us to go over into each of them one by one , which isn't  feasible
# 
# 
# * "Twitter" column also can't be scraped according to their new policy , tried multiple APIs and libraries but none of them worked , even twitter's free tier API is useless
#  
# 
# * "Acquisition ID" is just used to link between files , and we can do that with the company's name
# 

In [ ]:


acquired = acquired.drop(
    ["CrunchBase Profile", "Homepage", "Twitter", "Acquisitions ID", "API"], axis=1
)
acquiring = acquiring.drop(
    ["CrunchBase Profile", "Homepage", "Twitter", "Acquisitions ID", "API"], axis=1
)
founders = founders.drop("CrunchBase Profile", axis=1)

In [ ]:


acquired["Age on acquisition"] = (
    acquired["Year of acquisition announcement"] - acquired["Year Founded"]
)

In [ ]:


acquired = acquired.drop(["Year Founded", "Year of acquisition announcement"], axis=1)

In [ ]:


acquired = acquired.astype(
    {
        "Acquired by": "category",
        "City (HQ)": "category",
        "State / Region (HQ)": "category",
        "Country (HQ)": "category",
    }
)


# All these columns are probably related to the target column , so we will keep them for now

# Market categories contains multiple values , still not processed

In [ ]:


acquired.info()


# Dropping 'year of last update' of the number of employees , because we don't need it directly and can't use it in any way to pridct the current number

In [ ]:


acquiring = acquiring.drop("Number of Employees (year of last update)", axis=1)


# There are multiple 'NOT YET' in the IPO column , and the earliest the number the better it is , so we won't replace them with zero ,we will replace them with 2025 or anything larger

In [ ]:


acquiring["IPO"].value_counts()[:5]

In [ ]:


acquiring.loc[acquiring["IPO"] == "Not yet", "IPO"] = 2025  # 2025 is debatable
acquiring.loc[acquiring["IPO"].isna(), "IPO"] = 2025  # 2025 is debatable


# Idea for acquiring companies: calculate the average price paid for all acquired companies

# how to categorize multiple values in the same cell?

In [ ]:


acquiring["Market Categories"][:5]

In [ ]:


acquiring = acquiring.astype(
    {
        "City (HQ)": "category",
        "State / Region (HQ)": "category",
        "Country (HQ)": "category",
        "IPO": "float",
    }
)

In [ ]:


flattened = [x for item in acquiring["Board Members"].dropna() for x in item.split(",")]

In [ ]:


pd.Series(flattened).nunique()

In [ ]:


len(np.intersect1d(founders["Name"], flattened))


# Some of the board members are in the founders df , so we won't drop them for now

In [ ]:


acquiring.info()

In [ ]:


founders["Companies"].value_counts()[:5]

In [ ]:


founders["Role"].value_counts()

In [ ]:


founders = founders.astype({"Role": "category", "Companies": "category"})


# The image of the founder doesn't affect anything at all ... DROPPED

In [ ]:


founders = founders.drop("Image", axis=1)


# Ready

In [ ]:


founders.info()


# * The specific date which the deal was announced on doesn't matter , what matters is the year so the model can know that inflation affects the price
# * The ID doesn't add any new info
# * The News and News link don't add any info or details about the acquisition

In [ ]:


acquisitions = acquisitions.drop(
    ["Deal announced on", "Acquisitions ID", "News", "News Link"], axis=1
)

In [ ]:


acquisitions["Status"].value_counts()

In [ ]:


acquisitions["Terms"].value_counts()

In [ ]:


acquisitions = acquisitions.astype(
    {
        "Terms": "category",
        "Status": "category",
    }
)

In [ ]:


acquisitions.info()


# ### Spliting each multi-valued category to an array of categories

In [ ]:


def SplitMultiValuedColumn(df,label):
    df[label] = [
        [value.lstrip().rstrip() if type(value) == str else value for value in str(values).split(',')] for values in df[label]
    ]
    df[label].info

In [ ]:


def getUniqueLabels(df,label):
    uniqueLabels = []
    for labels in df[label]:
        for label in labels:
            if [label] not in uniqueLabels:
                uniqueLabels.append([label])
    return np.ravel(uniqueLabels)

In [ ]:


def encodeMultiValuedCategory(df , label : str, categories = np.array([])):
    le = preprocessing.LabelEncoder()
    SplitMultiValuedColumn(df,label)
    if not categories.any():
        categories = getUniqueLabels(df,label)
    le.fit(np.asarray(categories))
    df[label] = [le.transform(np.asarray(values)) for values in df[label]]
    return le.classes_

def encodeCategory(df , label : str, categories = np.array([])):
    le = preprocessing.LabelEncoder()
    if not categories.any():
        categories = df[label]
    le.fit(categories)
    df[label] = le.transform(df[label])
    return le.classes_


# ##### encoding data in Aquisitions

In [ ]:


multiValuedLabels = ["Terms","Acquiring Company","Acquired Company"]
for label in multiValuedLabels:
    print(encodeMultiValuedCategory(acquisitions,label)[:5])

In [ ]:


acquisitions["Terms"][:5]

In [ ]:


encodeCategory(acquisitions,"Status")

In [ ]:


acquisitions[:3]


# ##### encoding data in Founders

In [ ]:


founders.info

In [ ]:


foundersNames = encodeCategory(founders,"Name")

In [ ]:


foundersNames[:3]

In [ ]:


foundersRoles = encodeMultiValuedCategory(founders,"Role")

In [ ]:


foundersRoles


# ##### encoding data in Aquiring

In [ ]:


acquiringCompanies = encodeMultiValuedCategory(acquiring,"Acquiring Company")
acquiredCompanies = encodeMultiValuedCategory(acquiring,"Acquired Companies")

categories = ["City (HQ)","State / Region (HQ)","Country (HQ)"]
multiValuedCategories = ["Market Categories","Board Members"]
# saving acquiring companies names so i can encode the same column in different dataFrame with the same values
# and there is multiple shared columns in different dataframes that needs this operation

for c in categories:
    print(encodeCategory(acquiring,c)[:3])

for c in multiValuedCategories:
    print(encodeMultiValuedCategory(acquiring,c)[:3])

In [ ]:


encodeMultiValuedCategory(acquiring,"Founders",foundersNames)[:2]

In [ ]:


acquiring[:2]


# ##### encoding data in Acquired

In [ ]:


encodeCategory(acquired,"Company",acquiredCompanies)

In [ ]:


# i need to make global variables for this categories to get the unique from merging acquiring and acuired data
# something getUniqueValues(acquired["City (HQ)"] + acquiring["City (HQ)"])

for c in ["City (HQ)","State / Region (HQ)","Country (HQ)","Acquired by"]:
    print(encodeCategory(acquired,c)[:3])
encodeMultiValuedCategory(acquired,"Market Categories")[:3]

In [ ]:


acquired[:5]

numeric_df = acquired.select_dtypes(include='number')
correlations = numeric_df.drop("Price", axis=1).apply(lambda x: x.corr(numeric_df["Price"], method='kendall'))

#correlations = acquired.drop("Price", axis=1).apply(lambda x: x.corr(acquired["Price"], method='kendall'))
print(correlations)